In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import json
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.preprocessing import OneHotEncoder
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

In [2]:
#import training set
df = pd.read_csv('/Users/Nathan/Springboard/dota2.training.data.csv')

df.head()

,match_id,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_hero,...,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,dire_total_gold,dire_total_xp,dire_total_kills,dire_total_deaths,radiant_win
0,1,44,1,1448,1163,14,0,0,7,97,...,0,2,0,0,NaN,4348,6419,0,0,1
1,2,77,1,1311,1809,31,0,0,6,47,...,2,1,3,2,-9.0,3989,5115,0,2,0
2,3,66,2,1477,1367,25,0,0,7,22,...,4,2,2,0,18.0,5096,6007,1,1,1
3,4,5,0,697,807,8,0,0,10,53,...,3,1,0,0,NaN,4907,5937,1,1,0
4,5,102,1,962,1087,11,0,0,6,98,...,2,4,1,0,NaN,5471,5931,2,1,0


In [3]:
#print column names
for col in df.columns:
    print(col)

match_id
r1_hero
r1_level
r1_xp
r1_gold
r1_lh
r1_kills
r1_deaths
r1_items
r2_hero
r2_level
r2_xp
r2_gold
r2_lh
r2_kills
r2_deaths
r2_items
r3_hero
r3_level
r3_xp
r3_gold
r3_lh
r3_kills
r3_deaths
r3_items
r4_hero
r4_level
r4_xp
r4_gold
r4_lh
r4_kills
r4_deaths
r4_items
r5_hero
r5_level
r5_xp
r5_gold
r5_lh
r5_kills
r5_deaths
r5_items
d1_hero
d1_level
d1_xp
d1_gold
d1_lh
d1_kills
d1_deaths
d1_items
d2_hero
d2_level
d2_xp
d2_gold
d2_lh
d2_kills
d2_deaths
d2_items
d3_hero
d3_level
d3_xp
d3_gold
d3_lh
d3_kills
d3_deaths
d3_items
d4_hero
d4_level
d4_xp
d4_gold
d4_lh
d4_kills
d4_deaths
d4_items
d5_hero
d5_level
d5_xp
d5_gold
d5_lh
d5_kills
d5_deaths
d5_items
first_blood_time
first_blood_team
first_blood_player1
first_blood_player2
radiant_bottle_time
radiant_courier_time
radiant_flying_courier_time
radiant_tpscroll_count
radiant_boots_count
radiant_ward_observer_count
radiant_ward_sentry_count
radiant_first_ward_time
radiant_total_gold
radiant_total_xp
radiant_total_kills
radiant_total_deaths


In [4]:
#load in heroes JSON file and convert to df
j = json.load((open('/Users/Nathan/Springboard/hero_names.json')))

heroes_df = pd.read_json('/Users/Nathan/Springboard/hero_names.json')

heroes_df.head()

,npc_dota_hero_antimage,npc_dota_hero_axe,npc_dota_hero_bane,npc_dota_hero_bloodseeker,npc_dota_hero_crystal_maiden,npc_dota_hero_drow_ranger,npc_dota_hero_earthshaker,npc_dota_hero_juggernaut,npc_dota_hero_mirana,npc_dota_hero_morphling,...,npc_dota_hero_terrorblade,npc_dota_hero_phoenix,npc_dota_hero_oracle,npc_dota_hero_winter_wyvern,npc_dota_hero_arc_warden,npc_dota_hero_monkey_king,npc_dota_hero_dark_willow,npc_dota_hero_pangolier,npc_dota_hero_grimstroke,npc_dota_hero_mars
agi_gain,3.2,2.2,2.6,3.5,1.6,2.8,1.4,2.8,3.7,4.3,...,4.8,1.3,1.7,1.9,2.4,3.7,1.6,3.2,1.9,1.9
attack_range,150,150,400,150,600,625,150,150,630,350,...,150,500,620,425,625,300,475,150,550,250
attack_rate,1.4,1.7,1.7,1.7,1.7,1.7,1.7,1.4,1.7,1.5,...,1.5,1.7,1.4,1.7,1.7,1.7,1.3,1.7,1.7,1.8
attack_type,Melee,Melee,Ranged,Melee,Ranged,Ranged,Melee,Melee,Ranged,Ranged,...,Melee,Ranged,Ranged,Ranged,Ranged,Melee,Ranged,Melee,Ranged,Melee
base_agi,24,20,23,24,16,29,12,36,18,24,...,22,12,15,16,15,22,18,18,18,20


In [5]:
#transpose heroes df shape so that each row represents a single hero
hero_df = heroes_df.transpose()

hero_df.head()

,agi_gain,attack_range,attack_rate,attack_type,base_agi,base_armor,base_attack_max,base_attack_min,base_health,base_health_regen,...,int_gain,legs,localized_name,move_speed,name,primary_attr,projectile_speed,roles,str_gain,turn_rate
npc_dota_hero_antimage,3.2,150,1.4,Melee,24,-1,33,29,200,0.25,...,1.8,2,Anti-Mage,310,npc_dota_hero_antimage,agi,0,"[Carry, Escape, Nuker]",1.3,0.5
npc_dota_hero_axe,2.2,150,1.7,Melee,20,-2,28,24,200,2.75,...,1.6,2,Axe,295,npc_dota_hero_axe,str,900,"[Initiator, Durable, Disabler, Jungler]",3.4,0.6
npc_dota_hero_bane,2.6,400,1.7,Ranged,23,1,41,35,200,None,...,2.6,4,Bane,305,npc_dota_hero_bane,int,900,"[Support, Disabler, Nuker, Durable]",2.6,0.6
npc_dota_hero_bloodseeker,3.5,150,1.7,Melee,24,0,39,33,200,None,...,1.7,2,Bloodseeker,295,npc_dota_hero_bloodseeker,agi,900,"[Carry, Disabler, Jungler, Nuker, Initiator]",2.7,0.5
npc_dota_hero_crystal_maiden,1.6,600,1.7,Ranged,16,0,36,30,200,None,...,3.3,2,Crystal Maiden,275,npc_dota_hero_crystal_maiden,int,900,"[Support, Disabler, Nuker, Jungler]",2.2,0.5


In [6]:
#placeholder definition as variations of this will be used often
all_heroes = df[['r1_hero', 'r2_hero','r3_hero','r4_hero','r5_hero','d1_hero','d2_hero','d3_hero','d4_hero','d5_hero']]

In [7]:
#calculate radiant team hero frequency
r_heroes = df[['r1_hero', 'r2_hero','r3_hero','r4_hero','r5_hero']]

r_hero_freq = r_heroes.apply(pd.value_counts).sum(axis=1,skipna = True)

r_hero_freq.head()

1    13526
2    10680
3     5932
4     7153
5    14123
dtype: int64

In [8]:
#radiant team hero frequency with win
r_heroes_with_win = df[['r1_hero', 'r2_hero','r3_hero','r4_hero','r5_hero','radiant_win']]

r_heroes_win_only = r_heroes_with_win[r_heroes_with_win['radiant_win'] > 0]

r_heroes_win_only = r_heroes_win_only.drop(columns='radiant_win').apply(pd.value_counts).sum(axis=1,skipna = True)

r_heroes_win_only.head()

1    6736
2    4981
3    2881
4    3329
5    7922
dtype: int64

In [9]:
#radiant team hero win rate
r_heroes_win_per = r_heroes_win_only.divide(r_hero_freq)

In [10]:
#calculate dire team hero frequency
d_heroes = df[['d1_hero','d2_hero','d3_hero','d4_hero','d5_hero']]
d_hero_freq = d_heroes.apply(pd.value_counts).sum(axis=1,skipna = True)
d_hero_freq.head()

1    13864
2    10565
3     6099
4     7050
5    14061
dtype: int64

In [11]:
#dire team hero frequency with win
d_heroes_with_win = df[['d1_hero','d2_hero','d3_hero','d4_hero','d5_hero','radiant_win']]

d_heroes_win_only = d_heroes_with_win[d_heroes_with_win['radiant_win'] == 0]

d_heroes_win_only = d_heroes_win_only.drop(columns='radiant_win').apply(pd.value_counts).sum(axis=1,skipna = True)

d_heroes_win_only.head()

1    6459
2    4674
3    2836
4    3075
5    7441
dtype: int64

In [12]:
#dire team hero win rate
d_heroes_win_per = d_heroes_win_only.divide(d_hero_freq)

In [13]:
d_heroes_win_per.head()

1    0.465883
2    0.442404
3    0.464994
4    0.436170
5    0.529194
dtype: float64

In [14]:
r_heroes_win_per.head()

1    0.498004
2    0.466386
3    0.485671
4    0.465399
5    0.560929
dtype: float64

In [15]:
#average radiant and dire team hero win rates for all hero win rate
all_heroes_win_per = ((d_heroes_win_per + r_heroes_win_per)/2)*100

all_heroes_win_per.sort_values(ascending=False).head()

57     59.731539
67     58.812932
102    58.549115
42     57.560472
36     57.212145
dtype: float64

In [16]:
#radiant team win rate
df['radiant_win'].mean()

0.5117442990984621

In [17]:
#mean total gold by radiant teams with win
r_gold_win = df[['radiant_total_gold','radiant_win' ]]


radiant_mean_gold_if_win = r_gold_win[r_gold_win['radiant_win'] ==1].mean()

radiant_mean_gold_if_lose = r_gold_win[r_gold_win['radiant_win'] == 0].mean()

In [18]:
#mean total gold by dire teams with win
d_gold_win = df[['dire_total_gold','radiant_win' ]]

dire_mean_gold_if_win = d_gold_win[d_gold_win['radiant_win'] ==0].mean()

dire_mean_gold_if_lose = d_gold_win[d_gold_win['radiant_win'] ==1].mean()

In [19]:
mean_gold_if_win = (radiant_mean_gold_if_win[0] + dire_mean_gold_if_win[0])/2
mean_gold_if_win

5657.015217922237

In [20]:
mean_gold_if_lose = (radiant_mean_gold_if_lose[0] + dire_mean_gold_if_lose[0])/2
mean_gold_if_lose

5361.932958443441

In [21]:
#mean total xp by radiant teams with win
r_xp_win = df[['radiant_total_xp','radiant_win' ]]
radiant_mean_xp_if_win = r_xp_win[r_xp_win['radiant_win'] == 1].mean()
radiant_mean_xp_if_lose = r_xp_win[r_xp_win['radiant_win'] == 0].mean()

In [22]:
#mean total gold by dire teams with win
d_xp_win = df[['dire_total_xp','radiant_win' ]]
dire_mean_xp_if_win = d_xp_win[d_xp_win['radiant_win'] == 0].mean()
dire_mean_xp_if_lose = d_xp_win[d_xp_win['radiant_win'] == 1].mean()

In [23]:
mean_xp_if_win = (radiant_mean_xp_if_win[0] + dire_mean_xp_if_win[0])/2
mean_xp_if_win

6358.5066970796925

In [24]:
mean_xp_if_lose = (radiant_mean_xp_if_lose[0] + dire_mean_xp_if_lose[0])/2
mean_xp_if_lose

6146.367340016961

In [25]:
#mean total items by radiant teams with win
r_items_win = df[['r1_items','r2_items','r3_items','r4_items','r5_items','radiant_win']]
radiant_mean_items_if_win = r_items_win[r_items_win['radiant_win'] == 1].mean()
radiant_mean_items_if_lose = r_items_win[r_items_win['radiant_win'] == 0].mean()

In [26]:
#mean total items by dire teams with win
d_items_win = df[['d1_items','d2_items','d3_items','d4_items','d5_items','radiant_win']]
dire_mean_items_if_win = d_items_win[d_items_win['radiant_win'] == 0].mean()
dire_mean_items_if_lose = d_items_win[d_items_win['radiant_win'] == 1].mean()

In [27]:
mean_items_if_win = (radiant_mean_items_if_win[0] + dire_mean_items_if_win[0])/2
mean_items_if_win

7.756156597911167

In [28]:
mean_items_if_lose = (radiant_mean_items_if_lose[0] + dire_mean_items_if_lose[0])/2
mean_items_if_lose

7.549559562842598

In [29]:
#slice radiant time at first bottle item purchase with win, drop na to subset matches with radiant bottle purchase only
r_bottle = df[['radiant_bottle_time', 'radiant_win']].dropna()

In [30]:
#mean time of radiant bottle purchase with win vs. loss
r_bottle.groupby(['radiant_win']).mean()

,radiant_bottle_time
radiant_win,
0,118.047764
1,114.345904


In [31]:
#count of matches won by radiant team with bottle purchase
r_bottle_win = r_bottle.groupby(['radiant_win']).size()
r_bottle_win

radiant_win
0    64275
1    67802
dtype: int64

In [32]:
#radiant win rate with bottle item
r_bottle_win_per = r_bottle_win[1] / (r_bottle_win[0] + r_bottle_win[1])
r_bottle_win_per

0.5133520597833082

In [33]:
#slice dire time at first bottle item purchase with win, drop na to subset matches with dire bottle purchase only
d_bottle = df[['dire_bottle_time', 'radiant_win']].dropna()

In [34]:
#mean time of dire bottle purchase with win vs. loss
d_bottle.groupby(['radiant_win']).mean()

,dire_bottle_time
radiant_win,
0,115.298575
1,117.918082


In [35]:
#count of matches won by dire team with bottle purchase
d_bottle_win = d_bottle.groupby(['radiant_win']).size()
d_bottle_win

radiant_win
0    64627
1    66982
dtype: int64

In [36]:
#dire win rate with bottle item
d_bottle_win_per = d_bottle_win[0] / (d_bottle_win[0] + d_bottle_win[1])
d_bottle_win_per

0.49105304348486806

In [37]:
#overall bottle item win rate
bottle_win_per = (r_bottle_win_per + d_bottle_win_per)/2
bottle_win_per

0.5022025516340881

In [38]:
#slice radiant time at first courier item purchase with win, drop na to subset matches with radiant courier purchase only
r_courier = df[['radiant_courier_time', 'radiant_win']].dropna()
r_no_courier = df[['radiant_courier_time', 'radiant_win']]
r_courier.groupby(['radiant_win']).mean()

,radiant_courier_time
radiant_win,
0,-52.473356
1,-55.737009


In [39]:
r_no_courier["radiant_courier_time"][~r_no_courier["radiant_courier_time"].isna()]=-1
r_no_courier.fillna(1,inplace=True)

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
//anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:8682: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
//anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3325: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
//anaconda

In [42]:
r_no_courier.groupby(['radiant_win']).mean()

,radiant_courier_time
radiant_win,
0,-0.925282
1,-0.934411


In [43]:
#count of matches won by radiant team with courier purchase
r_courier_win = r_courier.groupby(['radiant_win']).size()
r_courier_win

radiant_win
0    85084
1    89600
dtype: int64

In [44]:
#radiant win rate with courier item
r_courier_win_per = r_courier_win[1] / (r_courier_win[0] + r_courier_win[1])
r_courier_win_per

0.512926198163541

In [45]:
#slice dire time at first courier item purchase with win, drop na to subset matches with dire courier purchase only
d_courier = df[['dire_courier_time', 'radiant_win']].dropna()
d_courier.groupby(['radiant_win']).mean()

,dire_courier_time
radiant_win,
0,-56.112380
1,-52.643638


In [46]:
#count of matches won by dire team with courier purchase
d_courier_win = d_courier.groupby(['radiant_win']).size()
d_courier_win

radiant_win
0    85567
1    89308
dtype: int64

In [47]:
#dire win rate with courier item
d_courier_win_per = d_courier_win[0] / (d_courier_win[0] + d_courier_win[1])
d_courier_win_per

0.4893037884203002

In [48]:
#overall courier item win rate
courier_win_per = (r_courier_win_per + d_courier_win_per)/2
courier_win_per

0.5011149932919206

In [49]:
#slice radiant time at first flying courier item purchase with win, drop na to subset matches with radiant flying courier purchase only
r_fcourier = df[['radiant_flying_courier_time', 'radiant_win']].dropna()
r_fcourier.groupby(['radiant_win']).mean()

,radiant_flying_courier_time
radiant_win,
0,223.59853
1,222.45330


In [50]:
#count of matches won by radiant team with flying courier purchase
r_fcourier_win = r_fcourier.groupby(['radiant_win']).size()
r_fcourier_win

radiant_win
0    39318
1    46124
dtype: int64

In [51]:
#radiant win rate with flying courier item
r_fcourier_win_per = r_fcourier_win[1] / (r_fcourier_win[0] + r_fcourier_win[1])*100
r_fcourier_win_per

53.98281875424264

In [52]:
#slice dire time at first flying courier item purchase with win, drop na to subset matches with dire flying courier purchase only
d_fcourier = df[['dire_flying_courier_time', 'radiant_win']].dropna()
d_fcourier.groupby(['radiant_win']).mean()

,dire_flying_courier_time
radiant_win,
0,221.671488
1,222.465330


In [53]:
#count of matches won by dire team with flying courier purchase
d_fcourier_win = d_fcourier.groupby(['radiant_win']).size()
d_fcourier_win

radiant_win
0    46187
1    43107
dtype: int64

In [54]:
#dire win rate with flying courier item
d_fcourier_win_per = d_fcourier_win[0] / (d_fcourier_win[0] + d_fcourier_win[1])*100
d_fcourier_win_per

51.72463995341232

In [55]:
#overall flying courier item win rate
fcourier_win_per = (r_fcourier_win_per + d_fcourier_win_per)/2
fcourier_win_per

52.85372935382748

In [56]:
#slice radiant time at first tpscroll item purchase with win, drop na to subset matches with radiant tpscroll purchase only
r_scroll = df[['radiant_tpscroll_count', 'radiant_win']].dropna()
r_scroll.groupby(['radiant_win']).mean()

,radiant_tpscroll_count
radiant_win,
0,2.895255
1,2.622887


In [57]:
#count of matches won by radiant team with tpscroll purchase
r_scroll_win = r_scroll.groupby(['radiant_win']).size()
r_scroll_win

radiant_win
0    88386
1    92638
dtype: int64

In [58]:
#radiant win rate with tpscroll item
r_scroll_win_per = r_scroll_win[1] / (r_scroll_win[0] + r_scroll_win[1])*100
r_scroll_win_per

51.17442990984621

In [59]:
#slice dire time at first tpscroll item purchase with win, drop na to subset matches with dire tpscroll purchase only
d_scroll = df[['dire_tpscroll_count', 'radiant_win']].dropna()
d_scroll.groupby(['radiant_win']).mean()

,dire_tpscroll_count
radiant_win,
0,2.611115
1,2.828979


In [60]:
#count of matches won by dire team with tpscroll purchase
d_scroll_win = d_scroll.groupby(['radiant_win']).size()
d_scroll_win

radiant_win
0    88386
1    92638
dtype: int64

In [61]:
#dire win rate with tpscroll item
d_scroll_win_per = d_scroll_win[0] / (d_scroll_win[0] + d_scroll_win[1])*100
d_scroll_win_per

48.82557009015379

In [62]:
#overall tpscroll item win rate
scroll_win_per = (r_scroll_win_per + d_scroll_win_per)/2
scroll_win_per

50.0

In [63]:
#slice radiant time at first boots item purchase with win, drop na to subset matches with radiant boots purchase only
r_boots = df[['radiant_boots_count', 'radiant_win']].dropna()
r_boots.groupby(['radiant_win']).mean()

,radiant_boots_count
radiant_win,
0,3.070011
1,3.235217


In [64]:
#count of matches won by radiant team with boots purchase
r_boots_win = r_boots.groupby(['radiant_win']).size()
r_boots_win

radiant_win
0    88386
1    92638
dtype: int64

In [65]:
#radiant win rate with boots item
r_boots_win_per = r_boots_win[1] / (r_boots_win[0] + r_boots_win[1])*100
r_boots_win_per

51.17442990984621

In [66]:
#slice dire time at first boots item purchase with win, drop na to subset matches with dire boots purchase only
d_boots = df[['dire_boots_count', 'radiant_win']].dropna()
d_boots.groupby(['radiant_win']).mean()

,dire_boots_count
radiant_win,
0,3.260777
1,3.097487


In [67]:
#count of matches won by dire team with boots purchase
d_boots_win = d_boots.groupby(['radiant_win']).size()
d_boots_win

radiant_win
0    88386
1    92638
dtype: int64

In [68]:
#dire win rate with boots item
d_boots_win_per = d_boots_win[0] / (d_boots_win[0] + d_boots_win[1])*100
d_boots_win_per

48.82557009015379

In [69]:
#overall boots item win rate
boots_win_per = (r_boots_win_per + d_boots_win_per)/2
boots_win_per

50.0

In [70]:
#slice radiant time at first ward observer item purchase with win, drop na to subset matches with radian ward observer purchase only
r_ward_obs = df[['radiant_ward_observer_count', 'radiant_win']].dropna()
r_ward_obs.groupby(['radiant_win']).mean()

,radiant_ward_observer_count
radiant_win,
0,1.651913
1,1.735659


In [71]:
#count of matches won by radiant team with ward observer purchase
r_ward_obs_win = r_ward_obs.groupby(['radiant_win']).size()
r_ward_obs_win
#resulting overall win % will be 50%

radiant_win
0    88386
1    92638
dtype: int64

In [72]:
#slice radiant time at first ward sentry item purchase with win, drop na to subset matches with radiant ward sentry purchase only
r_ward_sen = df[['radiant_ward_sentry_count', 'radiant_win']].dropna()
r_ward_sen.groupby(['radiant_win']).mean()

,radiant_ward_sentry_count
radiant_win,
0,0.237560
1,0.245051


In [73]:
#count of matches won by radiant team with ward sentry purchase
r_ward_sen_win = r_ward_sen.groupby(['radiant_win']).size()
r_ward_sen_win
#resulting overall win % will be 50%

radiant_win
0    88386
1    92638
dtype: int64

In [74]:
#slice radiant time at first ward item purchase with win, drop na to subset matches with radiant ward purchase only
r_ward = df[['radiant_first_ward_time', 'radiant_win']].dropna()
r_ward.groupby(['radiant_win']).mean()

,radiant_first_ward_time
radiant_win,
0,27.283784
1,24.359662


In [75]:
#count of matches won by radiant team with ward purchase
r_ward_win = r_ward.groupby(['radiant_win']).size()
r_ward_win

radiant_win
0    72527
1    78810
dtype: int64

In [76]:
#radiant win rate with ward item
r_ward_win_per = r_ward_win[1] / (r_ward_win[0] + r_ward_win[1])*100
r_ward_win_per

52.07583076180974

In [77]:
#slice dire time at first ward sentry item purchase with win, drop na to subset matches with dire ward sentry purchase only
d_ward = df[['dire_first_ward_time', 'radiant_win']].dropna()
d_ward.groupby(['radiant_win']).mean()

,dire_first_ward_time
radiant_win,
0,23.886761
1,28.328938


In [78]:
#count of matches won by dire team with ward purchase
d_ward_win = d_ward.groupby(['radiant_win']).size()
d_ward_win

radiant_win
0    74895
1    75467
dtype: int64

In [79]:
#dire win rate with ward item
d_ward_win_per = d_ward_win[0] / (d_ward_win[0] + d_ward_win[1])*100
d_ward_win_per

49.80979236775249

In [80]:
#overall ward item win rate
ward_win_per = (r_ward_win_per + d_ward_win_per)/2
ward_win_per

50.94281156478112

In [81]:
df = pd.read_csv('/Users/Nathan/Springboard/dota2.training.data.csv')
df.head()

,match_id,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_hero,...,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,dire_total_gold,dire_total_xp,dire_total_kills,dire_total_deaths,radiant_win
0,1,44,1,1448,1163,14,0,0,7,97,...,0,2,0,0,NaN,4348,6419,0,0,1
1,2,77,1,1311,1809,31,0,0,6,47,...,2,1,3,2,-9.0,3989,5115,0,2,0
2,3,66,2,1477,1367,25,0,0,7,22,...,4,2,2,0,18.0,5096,6007,1,1,1
3,4,5,0,697,807,8,0,0,10,53,...,3,1,0,0,NaN,4907,5937,1,1,0
4,5,102,1,962,1087,11,0,0,6,98,...,2,4,1,0,NaN,5471,5931,2,1,0


In [82]:
hero_df = pd.read_json('/Users/Nathan/Springboard/hero_names.json')

In [83]:
hero_df = hero_df.transpose()
hero_df.head()

,agi_gain,attack_range,attack_rate,attack_type,base_agi,base_armor,base_attack_max,base_attack_min,base_health,base_health_regen,...,int_gain,legs,localized_name,move_speed,name,primary_attr,projectile_speed,roles,str_gain,turn_rate
npc_dota_hero_antimage,3.2,150,1.4,Melee,24,-1,33,29,200,0.25,...,1.8,2,Anti-Mage,310,npc_dota_hero_antimage,agi,0,"[Carry, Escape, Nuker]",1.3,0.5
npc_dota_hero_axe,2.2,150,1.7,Melee,20,-2,28,24,200,2.75,...,1.6,2,Axe,295,npc_dota_hero_axe,str,900,"[Initiator, Durable, Disabler, Jungler]",3.4,0.6
npc_dota_hero_bane,2.6,400,1.7,Ranged,23,1,41,35,200,None,...,2.6,4,Bane,305,npc_dota_hero_bane,int,900,"[Support, Disabler, Nuker, Durable]",2.6,0.6
npc_dota_hero_bloodseeker,3.5,150,1.7,Melee,24,0,39,33,200,None,...,1.7,2,Bloodseeker,295,npc_dota_hero_bloodseeker,agi,900,"[Carry, Disabler, Jungler, Nuker, Initiator]",2.7,0.5
npc_dota_hero_crystal_maiden,1.6,600,1.7,Ranged,16,0,36,30,200,None,...,3.3,2,Crystal Maiden,275,npc_dota_hero_crystal_maiden,int,900,"[Support, Disabler, Nuker, Jungler]",2.2,0.5


In [84]:
list_of_all_players = ['r1', 'r2','r3','r4','r5','d1','d2','d3','d4','d5']

In [85]:
for player in list_of_all_players:
    print('Joining data for player: '+ player)
    
    hero_local = hero_df.copy()
    hero_local.columns = player + '_' + hero_df.columns
    
    df = df.merge(hero_local, left_on=[player + '_hero'], right_on=[player + '_id'], how='left')

Joining data for player: r1
Joining data for player: r2
Joining data for player: r3
Joining data for player: r4
Joining data for player: r5
Joining data for player: d1
Joining data for player: d2
Joining data for player: d3
Joining data for player: d4
Joining data for player: d5


In [86]:
df.head()

,match_id,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_hero,...,d5_int_gain,d5_legs,d5_localized_name,d5_move_speed,d5_name,d5_primary_attr,d5_projectile_speed,d5_roles,d5_str_gain,d5_turn_rate
0,1,44,1,1448,1163,14,0,0,7,97,...,2.6,0,Medusa,275,npc_dota_hero_medusa,agi,1200,"[Carry, Disabler, Durable]",1.3,0.5
1,2,77,1,1311,1809,31,0,0,6,47,...,1.4,2,Drow Ranger,285,npc_dota_hero_drow_ranger,agi,1250,"[Carry, Disabler, Pusher]",1.9,0.7
2,3,66,2,1477,1367,25,0,0,7,22,...,1.5,2,Pudge,280,npc_dota_hero_pudge,str,0,"[Disabler, Initiator, Durable, Nuker]",4,0.7
3,4,5,0,697,807,8,0,0,10,53,...,1.5,0,Slardar,290,npc_dota_hero_slardar,str,0,"[Carry, Durable, Initiator, Disabler, Escape]",3.6,0.5
4,5,102,1,962,1087,11,0,0,6,98,...,1.8,2,Ember Spirit,305,npc_dota_hero_ember_spirit,agi,900,"[Carry, Escape, Nuker, Disabler, Initiator]",2.6,0.5


In [87]:
for col in df.columns:
    print(col)

match_id
r1_hero
r1_level
r1_xp
r1_gold
r1_lh
r1_kills
r1_deaths
r1_items
r2_hero
r2_level
r2_xp
r2_gold
r2_lh
r2_kills
r2_deaths
r2_items
r3_hero
r3_level
r3_xp
r3_gold
r3_lh
r3_kills
r3_deaths
r3_items
r4_hero
r4_level
r4_xp
r4_gold
r4_lh
r4_kills
r4_deaths
r4_items
r5_hero
r5_level
r5_xp
r5_gold
r5_lh
r5_kills
r5_deaths
r5_items
d1_hero
d1_level
d1_xp
d1_gold
d1_lh
d1_kills
d1_deaths
d1_items
d2_hero
d2_level
d2_xp
d2_gold
d2_lh
d2_kills
d2_deaths
d2_items
d3_hero
d3_level
d3_xp
d3_gold
d3_lh
d3_kills
d3_deaths
d3_items
d4_hero
d4_level
d4_xp
d4_gold
d4_lh
d4_kills
d4_deaths
d4_items
d5_hero
d5_level
d5_xp
d5_gold
d5_lh
d5_kills
d5_deaths
d5_items
first_blood_time
first_blood_team
first_blood_player1
first_blood_player2
radiant_bottle_time
radiant_courier_time
radiant_flying_courier_time
radiant_tpscroll_count
radiant_boots_count
radiant_ward_observer_count
radiant_ward_sentry_count
radiant_first_ward_time
radiant_total_gold
radiant_total_xp
radiant_total_kills
radiant_total_deaths


# agi_gain mean by team, by win

In [88]:
agi_gain = df[['r1_agi_gain', 'r2_agi_gain','r3_agi_gain','r4_agi_gain','r5_agi_gain','d1_agi_gain','d2_agi_gain','d3_agi_gain','d4_agi_gain','d5_agi_gain','radiant_win']]

In [89]:
r_agi_gain = df[['r1_agi_gain', 'r2_agi_gain','r3_agi_gain','r4_agi_gain','r5_agi_gain','radiant_win']]

In [90]:
r_agi_gain[r_agi_gain['radiant_win'] ==1].sum(axis=1).mean()

11.505371445843007

In [91]:
r_agi_gain[r_agi_gain['radiant_win'] ==0].sum(axis=1).mean()

10.564699160500586

In [92]:
d_agi_gain = df[['d1_agi_gain','d2_agi_gain','d3_agi_gain','d4_agi_gain','d5_agi_gain','radiant_win']]

In [93]:
d_agi_gain[d_agi_gain['radiant_win'] ==1].sum(axis=1).mean()

11.548650661715433

In [94]:
d_agi_gain[d_agi_gain['radiant_win'] ==0].sum(axis=1).mean()

10.50017649854046

# attack_range mean by team, by win

In [95]:
r_attack_range = df[['r1_attack_range', 'r2_attack_range','r3_attack_range','r4_attack_range','r5_attack_range','radiant_win']]

Radiant mean attack range with win

In [96]:
r_attack_range[r_attack_range['radiant_win'] ==1].sum(axis=1).mean()

1701.866221205121

Radiant mean attack range with loss

In [97]:
r_attack_range[r_attack_range['radiant_win'] ==0].sum(axis=1).mean()

1722.2543728644807

In [98]:
d_attack_range = df[['d1_attack_range', 'd2_attack_range','d3_attack_range','d4_attack_range','d5_attack_range','radiant_win']]

Dire mean attack range with loss

In [99]:
d_attack_range[d_attack_range['radiant_win'] ==1].sum(axis=1).mean()

1724.3742632612966

Dire mean attack range with win

In [100]:
d_attack_range[d_attack_range['radiant_win'] ==0].sum(axis=1).mean()

1700.326974860272

In [103]:
all_heroes_win_per.head()

1    48.194335
2    45.439497
3    47.533260
4    45.078467
5    54.506160
dtype: float64

In [104]:
hero_namez = hero_df[['id', 'name']]

In [111]:
first_train.columns[first_train.isna().any()].tolist()

NameError: name 'first_train' is not defined

In [ ]:
first_train['first_blood_time'].fillna((first_train['first_blood_time'].mean()), inplace=True)

In [112]:
fbTeam = first_train[['first_blood_team', 'radiant_win']]

NameError: name 'first_train' is not defined

In [113]:
fbTeam.groupby(['first_blood_team']).count()

NameError: name 'fbTeam' is not defined

In [ ]:
first_train['radiant_bottle_time'].fillna((first_train['radiant_bottle_time'].mean()), inplace=True)

In [ ]:
first_train['radiant_courier_time'].fillna((first_train['radiant_courier_time'].mean()), inplace=True)

In [ ]:
first_train['radiant_flying_courier_time'].fillna((first_train['radiant_flying_courier_time'].mean()), inplace=True)

In [ ]:
first_train['radiant_first_ward_time'].fillna((first_train['radiant_first_ward_time'].mean()), inplace=True)

In [ ]:
first_train['dire_courier_time'].fillna((first_train['dire_courier_time'].mean()), inplace=True)

In [ ]:
first_train['dire_bottle_time'].fillna((first_train['dire_bottle_time'].mean()), inplace=True)

In [ ]:
first_train['dire_flying_courier_time'].fillna((first_train['dire_flying_courier_time'].mean()), inplace=True)

In [ ]:
first_train['dire_first_ward_time'].fillna((first_train['dire_first_ward_time'].mean()), inplace=True)

In [ ]:
first_train.columns.get_loc('radiant_win')

In [ ]:
df.to_csv(r'/Users/Nathan/Desktop/capstone_train.csv')

In [ ]:
for col in first_train.columns:
    print(col)

In [ ]:
first_train = first_train.drop(['r1_roles','r2_roles','r3_roles','r4_roles','r5_roles','d1_roles','d2_roles','d3_roles','d4_roles','d5_roles'], axis=1)



In [ ]:
for col in first_train.columns:
    print("'"+col+"'"+",")

In [114]:
for col_name in first_train.columns:
    if(first_train[col_name].dtype == 'object'):
        first_train[col_name]= first_train[col_name].astype('category')
        first_train[col_name] = first_train[col_name].cat.codes

NameError: name 'first_train' is not defined

# Creating Random Forest

In [ ]:
X=first_train[['match_id',
'r1_hero',
'r1_level',
'r1_xp',
'r1_gold',
'r1_lh',
'r1_kills',
'r1_deaths',
'r1_items',
'r2_hero',
'r2_level',
'r2_xp',
'r2_gold',
'r2_lh',
'r2_kills',
'r2_deaths',
'r2_items',
'r3_hero',
'r3_level',
'r3_xp',
'r3_gold',
'r3_lh',
'r3_kills',
'r3_deaths',
'r3_items',
'r4_hero',
'r4_level',
'r4_xp',
'r4_gold',
'r4_lh',
'r4_kills',
'r4_deaths',
'r4_items',
'r5_hero',
'r5_level',
'r5_xp',
'r5_gold',
'r5_lh',
'r5_kills',
'r5_deaths',
'r5_items',
'd1_hero',
'd1_level',
'd1_xp',
'd1_gold',
'd1_lh',
'd1_kills',
'd1_deaths',
'd1_items',
'd2_hero',
'd2_level',
'd2_xp',
'd2_gold',
'd2_lh',
'd2_kills',
'd2_deaths',
'd2_items',
'd3_hero',
'd3_level',
'd3_xp',
'd3_gold',
'd3_lh',
'd3_kills',
'd3_deaths',
'd3_items',
'd4_hero',
'd4_level',
'd4_xp',
'd4_gold',
'd4_lh',
'd4_kills',
'd4_deaths',
'd4_items',
'd5_hero',
'd5_level',
'd5_xp',
'd5_gold',
'd5_lh',
'd5_kills',
'd5_deaths',
'd5_items',
'first_blood_time',
'first_blood_team',
'first_blood_player1',
'first_blood_player2',
'radiant_bottle_time',
'radiant_courier_time',
'radiant_flying_courier_time',
'radiant_tpscroll_count',
'radiant_boots_count',
'radiant_ward_observer_count',
'radiant_ward_sentry_count',
'radiant_first_ward_time',
'radiant_total_gold',
'radiant_total_xp',
'radiant_total_kills',
'radiant_total_deaths',
'dire_bottle_time',
'dire_courier_time',
'dire_flying_courier_time',
'dire_tpscroll_count',
'dire_boots_count',
'dire_ward_observer_count',
'dire_ward_sentry_count',
'dire_first_ward_time',
'dire_total_gold',
'dire_total_xp',
'dire_total_kills',
'dire_total_deaths',
'r1_agi_gain',
'r1_attack_range',
'r1_attack_rate',
'r1_attack_type',
'r1_base_agi',
'r1_base_armor',
'r1_base_attack_max',
'r1_base_attack_min',
'r1_base_health',
'r1_base_int',
'r1_base_mana',
'r1_base_mana_regen',
'r1_base_mr',
'r1_base_str',
'r1_cm_enabled',
'r1_id',
'r1_int_gain',
'r1_legs',
'r1_move_speed',
'r1_primary_attr',
'r1_projectile_speed',
'r1_str_gain',
'r1_turn_rate',
'r2_agi_gain',
'r2_attack_range',
'r2_attack_rate',
'r2_attack_type',
'r2_base_agi',
'r2_base_armor',
'r2_base_attack_max',
'r2_base_attack_min',
'r2_base_health',
'r2_base_int',
'r2_base_mana',
'r2_base_mana_regen',
'r2_base_mr',
'r2_base_str',
'r2_cm_enabled',
'r2_id',
'r2_int_gain',
'r2_legs',
'r2_move_speed',
'r2_primary_attr',
'r2_projectile_speed',
'r2_str_gain',
'r2_turn_rate',
'r3_agi_gain',
'r3_attack_range',
'r3_attack_rate',
'r3_attack_type',
'r3_base_agi',
'r3_base_armor',
'r3_base_attack_max',
'r3_base_attack_min',
'r3_base_health',
'r3_base_int',
'r3_base_mana',
'r3_base_mana_regen',
'r3_base_mr',
'r3_base_str',
'r3_cm_enabled',
'r3_id',
'r3_int_gain',
'r3_legs',
'r3_move_speed',
'r3_primary_attr',
'r3_projectile_speed',
'r3_str_gain',
'r3_turn_rate',
'r4_agi_gain',
'r4_attack_range',
'r4_attack_rate',
'r4_attack_type',
'r4_base_agi',
'r4_base_armor',
'r4_base_attack_max',
'r4_base_attack_min',
'r4_base_health',
'r4_base_int',
'r4_base_mana',
'r4_base_mana_regen',
'r4_base_mr',
'r4_base_str',
'r4_cm_enabled',
'r4_id',
'r4_int_gain',
'r4_legs',
'r4_move_speed',
'r4_primary_attr',
'r4_projectile_speed',
'r4_str_gain',
'r4_turn_rate',
'r5_agi_gain',
'r5_attack_range',
'r5_attack_rate',
'r5_attack_type',
'r5_base_agi',
'r5_base_armor',
'r5_base_attack_max',
'r5_base_attack_min',
'r5_base_health',
'r5_base_int',
'r5_base_mana',
'r5_base_mana_regen',
'r5_base_mr',
'r5_base_str',
'r5_cm_enabled',
'r5_id',
'r5_int_gain',
'r5_legs',
'r5_move_speed',
'r5_primary_attr',
'r5_projectile_speed',
'r5_str_gain',
'r5_turn_rate',
'd1_agi_gain',
'd1_attack_range',
'd1_attack_rate',
'd1_attack_type',
'd1_base_agi',
'd1_base_armor',
'd1_base_attack_max',
'd1_base_attack_min',
'd1_base_health',
'd1_base_int',
'd1_base_mana',
'd1_base_mana_regen',
'd1_base_mr',
'd1_base_str',
'd1_cm_enabled',
'd1_id',
'd1_int_gain',
'd1_legs',
'd1_move_speed',
'd1_primary_attr',
'd1_projectile_speed',
'd1_str_gain',
'd1_turn_rate',
'd2_agi_gain',
'd2_attack_range',
'd2_attack_rate',
'd2_attack_type',
'd2_base_agi',
'd2_base_armor',
'd2_base_attack_max',
'd2_base_attack_min',
'd2_base_health',
'd2_base_int',
'd2_base_mana',
'd2_base_mana_regen',
'd2_base_mr',
'd2_base_str',
'd2_cm_enabled',
'd2_id',
'd2_int_gain',
'd2_legs',
'd2_move_speed',
'd2_primary_attr',
'd2_projectile_speed',
'd2_str_gain',
'd2_turn_rate',
'd3_agi_gain',
'd3_attack_range',
'd3_attack_rate',
'd3_attack_type',
'd3_base_agi',
'd3_base_armor',
'd3_base_attack_max',
'd3_base_attack_min',
'd3_base_health',
'd3_base_int',
'd3_base_mana',
'd3_base_mana_regen',
'd3_base_mr',
'd3_base_str',
'd3_cm_enabled',
'd3_id',
'd3_int_gain',
'd3_legs',
'd3_move_speed',
'd3_primary_attr',
'd3_projectile_speed',
'd3_str_gain',
'd3_turn_rate',
'd4_agi_gain',
'd4_attack_range',
'd4_attack_rate',
'd4_attack_type',
'd4_base_agi',
'd4_base_armor',
'd4_base_attack_max',
'd4_base_attack_min',
'd4_base_health',
'd4_base_int',
'd4_base_mana',
'd4_base_mana_regen',
'd4_base_mr',
'd4_base_str',
'd4_cm_enabled',
'd4_id',
'd4_int_gain',
'd4_legs',
'd4_move_speed',
'd4_primary_attr',
'd4_projectile_speed',
'd4_str_gain',
'd4_turn_rate',
'd5_agi_gain',
'd5_attack_range',
'd5_attack_rate',
'd5_attack_type',
'd5_base_agi',
'd5_base_armor',
'd5_base_attack_max',
'd5_base_attack_min',
'd5_base_health',
'd5_base_int',
'd5_base_mana',
'd5_base_mana_regen',
'd5_base_mr',
'd5_base_str',
'd5_cm_enabled',
'd5_id',
'd5_int_gain',
'd5_legs',
'd5_move_speed',
'd5_primary_attr',
'd5_projectile_speed',
'd5_str_gain',
'd5_turn_rate']]

In [ ]:
y=first_train['radiant_win']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

clf=RandomForestClassifier(n_estimators=100)

clf.fit(X_train,y_train)

In [ ]:
first_train.columns[first_train.isnull().any()]

In [ ]:
first_train = first_train.drop(['r1_base_health_regen', 'r2_base_health_regen', 'r3_base_health_regen',
       'r4_base_health_regen', 'r5_base_health_regen', 'd1_base_health_regen',
       'd2_base_health_regen', 'd3_base_health_regen', 'd4_base_health_regen',
       'd5_base_health_regen'],axis=1)

In [ ]:
y_pred=clf.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [ ]:
feature_imp = pd.Series(clf.feature_importances_,index=['match_id',
'r1_hero',
'r1_level',
'r1_xp',
'r1_gold',
'r1_lh',
'r1_kills',
'r1_deaths',
'r1_items',
'r2_hero',
'r2_level',
'r2_xp',
'r2_gold',
'r2_lh',
'r2_kills',
'r2_deaths',
'r2_items',
'r3_hero',
'r3_level',
'r3_xp',
'r3_gold',
'r3_lh',
'r3_kills',
'r3_deaths',
'r3_items',
'r4_hero',
'r4_level',
'r4_xp',
'r4_gold',
'r4_lh',
'r4_kills',
'r4_deaths',
'r4_items',
'r5_hero',
'r5_level',
'r5_xp',
'r5_gold',
'r5_lh',
'r5_kills',
'r5_deaths',
'r5_items',
'd1_hero',
'd1_level',
'd1_xp',
'd1_gold',
'd1_lh',
'd1_kills',
'd1_deaths',
'd1_items',
'd2_hero',
'd2_level',
'd2_xp',
'd2_gold',
'd2_lh',
'd2_kills',
'd2_deaths',
'd2_items',
'd3_hero',
'd3_level',
'd3_xp',
'd3_gold',
'd3_lh',
'd3_kills',
'd3_deaths',
'd3_items',
'd4_hero',
'd4_level',
'd4_xp',
'd4_gold',
'd4_lh',
'd4_kills',
'd4_deaths',
'd4_items',
'd5_hero',
'd5_level',
'd5_xp',
'd5_gold',
'd5_lh',
'd5_kills',
'd5_deaths',
'd5_items',
'first_blood_time',
'first_blood_team',
'first_blood_player1',
'first_blood_player2',
'radiant_bottle_time',
'radiant_courier_time',
'radiant_flying_courier_time',
'radiant_tpscroll_count',
'radiant_boots_count',
'radiant_ward_observer_count',
'radiant_ward_sentry_count',
'radiant_first_ward_time',
'radiant_total_gold',
'radiant_total_xp',
'radiant_total_kills',
'radiant_total_deaths',
'dire_bottle_time',
'dire_courier_time',
'dire_flying_courier_time',
'dire_tpscroll_count',
'dire_boots_count',
'dire_ward_observer_count',
'dire_ward_sentry_count',
'dire_first_ward_time',
'dire_total_gold',
'dire_total_xp',
'dire_total_kills',
'dire_total_deaths',
'r1_agi_gain',
'r1_attack_range',
'r1_attack_rate',
'r1_attack_type',
'r1_base_agi',
'r1_base_armor',
'r1_base_attack_max',
'r1_base_attack_min',
'r1_base_int',
'r1_base_mr',
'r1_base_str',
'r1_cm_enabled',
'r1_id',
'r1_int_gain',
'r1_legs',
'r1_move_speed',
'r1_primary_attr',
'r1_projectile_speed',
'r1_str_gain',
'r1_turn_rate',
'r2_agi_gain',
'r2_attack_range',
'r2_attack_rate',
'r2_attack_type',
'r2_base_agi',
'r2_base_armor',
'r2_base_attack_max',
'r2_base_attack_min',
'r2_base_int',
'r2_base_mr',
'r2_base_str',
'r2_cm_enabled',
'r2_id',
'r2_int_gain',
'r2_legs',
'r2_move_speed',
'r2_primary_attr',
'r2_projectile_speed',
'r2_str_gain',
'r2_turn_rate',
'r3_agi_gain',
'r3_attack_range',
'r3_attack_rate',
'r3_attack_type',
'r3_base_agi',
'r3_base_armor',
'r3_base_attack_max',
'r3_base_attack_min',
'r3_base_int',
'r3_base_mr',
'r3_base_str',
'r3_cm_enabled',
'r3_id',
'r3_int_gain',
'r3_legs',
'r3_move_speed',
'r3_primary_attr',
'r3_projectile_speed',
'r3_str_gain',
'r3_turn_rate',
'r4_agi_gain',
'r4_attack_range',
'r4_attack_rate',
'r4_attack_type',
'r4_base_agi',
'r4_base_armor',
'r4_base_attack_max',
'r4_base_attack_min',
'r4_base_int',
'r4_base_mr',
'r4_base_str',
'r4_cm_enabled',
'r4_id',
'r4_int_gain',
'r4_legs',
'r4_move_speed',
'r4_primary_attr',
'r4_projectile_speed',
'r4_str_gain',
'r4_turn_rate',
'r5_agi_gain',
'r5_attack_range',
'r5_attack_rate',
'r5_attack_type',
'r5_base_agi',
'r5_base_armor',
'r5_base_attack_max',
'r5_base_attack_min',
'r5_base_int',
'r5_base_mr',
'r5_base_str',
'r5_cm_enabled',
'r5_id',
'r5_int_gain',
'r5_legs',
'r5_move_speed',
'r5_primary_attr',
'r5_projectile_speed',
'r5_str_gain',
'r5_turn_rate',
'd1_agi_gain',
'd1_attack_range',
'd1_attack_rate',
'd1_attack_type',
'd1_base_agi',
'd1_base_armor',
'd1_base_attack_max',
'd1_base_attack_min',
'd1_base_int',
'd1_base_mr',
'd1_base_str',
'd1_cm_enabled',
'd1_id',
'd1_int_gain',
'd1_legs',
'd1_move_speed',
'd1_primary_attr',
'd1_projectile_speed',
'd1_str_gain',
'd1_turn_rate',
'd2_agi_gain',
'd2_attack_range',
'd2_attack_rate',
'd2_attack_type',
'd2_base_agi',
'd2_base_armor',
'd2_base_attack_max',
'd2_base_attack_min',
'd2_base_int',
'd2_base_mr',
'd2_base_str',
'd2_cm_enabled',
'd2_id',
'd2_int_gain',
'd2_legs',
'd2_move_speed',
'd2_primary_attr',
'd2_projectile_speed',
'd2_str_gain',
'd2_turn_rate',
'd3_agi_gain',
'd3_attack_range',
'd3_attack_rate',
'd3_attack_type',
'd3_base_agi',
'd3_base_armor',
'd3_base_attack_max',
'd3_base_attack_min',
'd3_base_int',
'd3_base_mr',
'd3_base_str',
'd3_cm_enabled',
'd3_id',
'd3_int_gain',
'd3_legs',
'd3_move_speed',
'd3_primary_attr',
'd3_projectile_speed',
'd3_str_gain',
'd3_turn_rate',
'd4_agi_gain',
'd4_attack_range',
'd4_attack_rate',
'd4_attack_type',
'd4_base_agi',
'd4_base_armor',
'd4_base_attack_max',
'd4_base_attack_min',
'd4_base_int',
'd4_base_mr',
'd4_base_str',
'd4_cm_enabled',
'd4_id',
'd4_int_gain',
'd4_legs',
'd4_move_speed',
'd4_primary_attr',
'd4_projectile_speed',
'd4_str_gain',
'd4_turn_rate',
'd5_agi_gain',
'd5_attack_range',
'd5_attack_rate',
'd5_attack_type',
'd5_base_agi',
'd5_base_armor',
'd5_base_attack_max',
'd5_base_attack_min',
'd5_base_int',
'd5_base_mr',
'd5_base_str',
'd5_cm_enabled',
'd5_id',
'd5_int_gain',
'd5_legs',
'd5_move_speed',
'd5_primary_attr',
'd5_projectile_speed',
'd5_str_gain',
'd5_turn_rate']).sort_values(ascending=False)

In [ ]:
feature_imp

removing features with 0 importance

In [ ]:
X=first_train[['match_id',
'r1_hero',
'r1_level',
'r1_xp',
'r1_gold',
'r1_lh',
'r1_kills',
'r1_deaths',
'r1_items',
'r2_hero',
'r2_level',
'r2_xp',
'r2_gold',
'r2_lh',
'r2_kills',
'r2_deaths',
'r2_items',
'r3_hero',
'r3_level',
'r3_xp',
'r3_gold',
'r3_lh',
'r3_kills',
'r3_deaths',
'r3_items',
'r4_hero',
'r4_level',
'r4_xp',
'r4_gold',
'r4_lh',
'r4_kills',
'r4_deaths',
'r4_items',
'r5_hero',
'r5_level',
'r5_xp',
'r5_gold',
'r5_lh',
'r5_kills',
'r5_deaths',
'r5_items',
'd1_hero',
'd1_level',
'd1_xp',
'd1_gold',
'd1_lh',
'd1_kills',
'd1_deaths',
'd1_items',
'd2_hero',
'd2_level',
'd2_xp',
'd2_gold',
'd2_lh',
'd2_kills',
'd2_deaths',
'd2_items',
'd3_hero',
'd3_level',
'd3_xp',
'd3_gold',
'd3_lh',
'd3_kills',
'd3_deaths',
'd3_items',
'd4_hero',
'd4_level',
'd4_xp',
'd4_gold',
'd4_lh',
'd4_kills',
'd4_deaths',
'd4_items',
'd5_hero',
'd5_level',
'd5_xp',
'd5_gold',
'd5_lh',
'd5_kills',
'd5_deaths',
'd5_items',
'first_blood_time',
'first_blood_team',
'first_blood_player1',
'first_blood_player2',
'radiant_bottle_time',
'radiant_courier_time',
'radiant_flying_courier_time',
'radiant_tpscroll_count',
'radiant_boots_count',
'radiant_ward_observer_count',
'radiant_ward_sentry_count',
'radiant_first_ward_time',
'radiant_total_gold',
'radiant_total_xp',
'radiant_total_kills',
'radiant_total_deaths',
'dire_bottle_time',
'dire_courier_time',
'dire_flying_courier_time',
'dire_tpscroll_count',
'dire_boots_count',
'dire_ward_observer_count',
'dire_ward_sentry_count',
'dire_first_ward_time',
'dire_total_gold',
'dire_total_xp',
'dire_total_kills',
'dire_total_deaths',
'r1_agi_gain',
'r1_attack_range',
'r1_attack_rate',
'r1_attack_type',
'r1_base_agi',
'r1_base_armor',
'r1_base_attack_max',
'r1_base_attack_min',
'r1_base_int',
'r1_base_mr',
'r1_base_str',
'r1_cm_enabled',
'r1_id',
'r1_int_gain',
'r1_legs',
'r1_move_speed',
'r1_primary_attr',
'r1_projectile_speed',
'r1_str_gain',
'r1_turn_rate',
'r2_agi_gain',
'r2_attack_range',
'r2_attack_rate',
'r2_attack_type',
'r2_base_agi',
'r2_base_armor',
'r2_base_attack_max',
'r2_base_attack_min',
'r2_base_int',
'r2_base_mr',
'r2_base_str',
'r2_cm_enabled',
'r2_id',
'r2_int_gain',
'r2_legs',
'r2_move_speed',
'r2_primary_attr',
'r2_projectile_speed',
'r2_str_gain',
'r2_turn_rate',
'r3_agi_gain',
'r3_attack_range',
'r3_attack_rate',
'r3_attack_type',
'r3_base_agi',
'r3_base_armor',
'r3_base_attack_max',
'r3_base_attack_min',
'r3_base_int',
'r3_base_mr',
'r3_base_str',
'r3_cm_enabled',
'r3_id',
'r3_int_gain',
'r3_legs',
'r3_move_speed',
'r3_primary_attr',
'r3_projectile_speed',
'r3_str_gain',
'r3_turn_rate',
'r4_agi_gain',
'r4_attack_range',
'r4_attack_rate',
'r4_attack_type',
'r4_base_agi',
'r4_base_armor',
'r4_base_attack_max',
'r4_base_attack_min',
'r4_base_int',
'r4_base_mr',
'r4_base_str',
'r4_cm_enabled',
'r4_id',
'r4_int_gain',
'r4_legs',
'r4_move_speed',
'r4_primary_attr',
'r4_projectile_speed',
'r4_str_gain',
'r4_turn_rate',
'r5_agi_gain',
'r5_attack_range',
'r5_attack_rate',
'r5_attack_type',
'r5_base_agi',
'r5_base_armor',
'r5_base_attack_max',
'r5_base_attack_min',
'r5_base_int',
'r5_base_mr',
'r5_base_str',
'r5_cm_enabled',
'r5_id',
'r5_int_gain',
'r5_legs',
'r5_move_speed',
'r5_primary_attr',
'r5_projectile_speed',
'r5_str_gain',
'r5_turn_rate',
'd1_agi_gain',
'd1_attack_range',
'd1_attack_rate',
'd1_attack_type',
'd1_base_agi',
'd1_base_armor',
'd1_base_attack_max',
'd1_base_attack_min',
'd1_base_int',
'd1_base_mr',
'd1_base_str',
'd1_cm_enabled',
'd1_id',
'd1_int_gain',
'd1_legs',
'd1_move_speed',
'd1_primary_attr',
'd1_projectile_speed',
'd1_str_gain',
'd1_turn_rate',
'd2_agi_gain',
'd2_attack_range',
'd2_attack_rate',
'd2_attack_type',
'd2_base_agi',
'd2_base_armor',
'd2_base_attack_max',
'd2_base_attack_min',
'd2_base_int',
'd2_base_mr',
'd2_base_str',
'd2_cm_enabled',
'd2_id',
'd2_int_gain',
'd2_legs',
'd2_move_speed',
'd2_primary_attr',
'd2_projectile_speed',
'd2_str_gain',
'd2_turn_rate',
'd3_agi_gain',
'd3_attack_range',
'd3_attack_rate',
'd3_attack_type',
'd3_base_agi',
'd3_base_armor',
'd3_base_attack_max',
'd3_base_attack_min',
'd3_base_int',
'd3_base_mr',
'd3_base_str',
'd3_cm_enabled',
'd3_id',
'd3_int_gain',
'd3_legs',
'd3_move_speed',
'd3_primary_attr',
'd3_projectile_speed',
'd3_str_gain',
'd3_turn_rate',
'd4_agi_gain',
'd4_attack_range',
'd4_attack_rate',
'd4_attack_type',
'd4_base_agi',
'd4_base_armor',
'd4_base_attack_max',
'd4_base_attack_min',
'd4_base_int',
'd4_base_mr',
'd4_base_str',
'd4_cm_enabled',
'd4_id',
'd4_int_gain',
'd4_legs',
'd4_move_speed',
'd4_primary_attr',
'd4_projectile_speed',
'd4_str_gain',
'd4_turn_rate',
'd5_agi_gain',
'd5_attack_range',
'd5_attack_rate',
'd5_attack_type',
'd5_base_agi',
'd5_base_armor',
'd5_base_attack_max',
'd5_base_attack_min',
'd5_base_int',
'd5_base_mr',
'd5_base_str',
'd5_cm_enabled',
'd5_id',
'd5_int_gain',
'd5_legs',
'd5_move_speed',
'd5_primary_attr',
'd5_projectile_speed',
'd5_str_gain',
'd5_turn_rate']]

In [ ]:
y=first_train['radiant_win']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

clf=RandomForestClassifier(n_estimators=100)

clf.fit(X_train,y_train)

In [ ]:
y_pred=clf.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [ ]:
feature_imp2 = pd.Series(clf.feature_importances_,index=['match_id',
'r1_hero',
'r1_level',
'r1_xp',
'r1_gold',
'r1_lh',
'r1_kills',
'r1_deaths',
'r1_items',
'r2_hero',
'r2_level',
'r2_xp',
'r2_gold',
'r2_lh',
'r2_kills',
'r2_deaths',
'r2_items',
'r3_hero',
'r3_level',
'r3_xp',
'r3_gold',
'r3_lh',
'r3_kills',
'r3_deaths',
'r3_items',
'r4_hero',
'r4_level',
'r4_xp',
'r4_gold',
'r4_lh',
'r4_kills',
'r4_deaths',
'r4_items',
'r5_hero',
'r5_level',
'r5_xp',
'r5_gold',
'r5_lh',
'r5_kills',
'r5_deaths',
'r5_items',
'd1_hero',
'd1_level',
'd1_xp',
'd1_gold',
'd1_lh',
'd1_kills',
'd1_deaths',
'd1_items',
'd2_hero',
'd2_level',
'd2_xp',
'd2_gold',
'd2_lh',
'd2_kills',
'd2_deaths',
'd2_items',
'd3_hero',
'd3_level',
'd3_xp',
'd3_gold',
'd3_lh',
'd3_kills',
'd3_deaths',
'd3_items',
'd4_hero',
'd4_level',
'd4_xp',
'd4_gold',
'd4_lh',
'd4_kills',
'd4_deaths',
'd4_items',
'd5_hero',
'd5_level',
'd5_xp',
'd5_gold',
'd5_lh',
'd5_kills',
'd5_deaths',
'd5_items',
'first_blood_time',
'first_blood_team',
'first_blood_player1',
'first_blood_player2',
'radiant_bottle_time',
'radiant_courier_time',
'radiant_flying_courier_time',
'radiant_tpscroll_count',
'radiant_boots_count',
'radiant_ward_observer_count',
'radiant_ward_sentry_count',
'radiant_first_ward_time',
'radiant_total_gold',
'radiant_total_xp',
'radiant_total_kills',
'radiant_total_deaths',
'dire_bottle_time',
'dire_courier_time',
'dire_flying_courier_time',
'dire_tpscroll_count',
'dire_boots_count',
'dire_ward_observer_count',
'dire_ward_sentry_count',
'dire_first_ward_time',
'dire_total_gold',
'dire_total_xp',
'dire_total_kills',
'dire_total_deaths',
'r1_agi_gain',
'r1_attack_range',
'r1_attack_rate',
'r1_attack_type',
'r1_base_agi',
'r1_base_armor',
'r1_base_attack_max',
'r1_base_attack_min',
'r1_base_int',
'r1_base_mr',
'r1_base_str',
'r1_cm_enabled',
'r1_id',
'r1_int_gain',
'r1_legs',
'r1_move_speed',
'r1_primary_attr',
'r1_projectile_speed',
'r1_str_gain',
'r1_turn_rate',
'r2_agi_gain',
'r2_attack_range',
'r2_attack_rate',
'r2_attack_type',
'r2_base_agi',
'r2_base_armor',
'r2_base_attack_max',
'r2_base_attack_min',
'r2_base_int',
'r2_base_mr',
'r2_base_str',
'r2_cm_enabled',
'r2_id',
'r2_int_gain',
'r2_legs',
'r2_move_speed',
'r2_primary_attr',
'r2_projectile_speed',
'r2_str_gain',
'r2_turn_rate',
'r3_agi_gain',
'r3_attack_range',
'r3_attack_rate',
'r3_attack_type',
'r3_base_agi',
'r3_base_armor',
'r3_base_attack_max',
'r3_base_attack_min',
'r3_base_int',
'r3_base_mr',
'r3_base_str',
'r3_cm_enabled',
'r3_id',
'r3_int_gain',
'r3_legs',
'r3_move_speed',
'r3_primary_attr',
'r3_projectile_speed',
'r3_str_gain',
'r3_turn_rate',
'r4_agi_gain',
'r4_attack_range',
'r4_attack_rate',
'r4_attack_type',
'r4_base_agi',
'r4_base_armor',
'r4_base_attack_max',
'r4_base_attack_min',
'r4_base_int',
'r4_base_mr',
'r4_base_str',
'r4_cm_enabled',
'r4_id',
'r4_int_gain',
'r4_legs',
'r4_move_speed',
'r4_primary_attr',
'r4_projectile_speed',
'r4_str_gain',
'r4_turn_rate',
'r5_agi_gain',
'r5_attack_range',
'r5_attack_rate',
'r5_attack_type',
'r5_base_agi',
'r5_base_armor',
'r5_base_attack_max',
'r5_base_attack_min',
'r5_base_int',
'r5_base_mr',
'r5_base_str',
'r5_cm_enabled',
'r5_id',
'r5_int_gain',
'r5_legs',
'r5_move_speed',
'r5_primary_attr',
'r5_projectile_speed',
'r5_str_gain',
'r5_turn_rate',
'd1_agi_gain',
'd1_attack_range',
'd1_attack_rate',
'd1_attack_type',
'd1_base_agi',
'd1_base_armor',
'd1_base_attack_max',
'd1_base_attack_min',
'd1_base_int',
'd1_base_mr',
'd1_base_str',
'd1_cm_enabled',
'd1_id',
'd1_int_gain',
'd1_legs',
'd1_move_speed',
'd1_primary_attr',
'd1_projectile_speed',
'd1_str_gain',
'd1_turn_rate',
'd2_agi_gain',
'd2_attack_range',
'd2_attack_rate',
'd2_attack_type',
'd2_base_agi',
'd2_base_armor',
'd2_base_attack_max',
'd2_base_attack_min',
'd2_base_int',
'd2_base_mr',
'd2_base_str',
'd2_cm_enabled',
'd2_id',
'd2_int_gain',
'd2_legs',
'd2_move_speed',
'd2_primary_attr',
'd2_projectile_speed',
'd2_str_gain',
'd2_turn_rate',
'd3_agi_gain',
'd3_attack_range',
'd3_attack_rate',
'd3_attack_type',
'd3_base_agi',
'd3_base_armor',
'd3_base_attack_max',
'd3_base_attack_min',
'd3_base_int',
'd3_base_mr',
'd3_base_str',
'd3_cm_enabled',
'd3_id',
'd3_int_gain',
'd3_legs',
'd3_move_speed',
'd3_primary_attr',
'd3_projectile_speed',
'd3_str_gain',
'd3_turn_rate',
'd4_agi_gain',
'd4_attack_range',
'd4_attack_rate',
'd4_attack_type',
'd4_base_agi',
'd4_base_armor',
'd4_base_attack_max',
'd4_base_attack_min',
'd4_base_int',
'd4_base_mr',
'd4_base_str',
'd4_cm_enabled',
'd4_id',
'd4_int_gain',
'd4_legs',
'd4_move_speed',
'd4_primary_attr',
'd4_projectile_speed',
'd4_str_gain',
'd4_turn_rate',
'd5_agi_gain',
'd5_attack_range',
'd5_attack_rate',
'd5_attack_type',
'd5_base_agi',
'd5_base_armor',
'd5_base_attack_max',
'd5_base_attack_min',
'd5_base_int',
'd5_base_mr',
'd5_base_str',
'd5_cm_enabled',
'd5_id',
'd5_int_gain',
'd5_legs',
'd5_move_speed',
'd5_primary_attr',
'd5_projectile_speed',
'd5_str_gain',
'd5_turn_rate']).sort_values(ascending=False)

In [336]:
feature_imp2

NameError: name 'feature_imp2' is not defined

In [ ]:
first_train[['radiant_win']].sum()

In [ ]:
pd.DataFrame(first_train)

In [115]:
train3 = pd.read_csv('/Users/Nathan/train3.csv')

In [116]:
train3['radiant_level'] = train3[['r1_level','r2_level','r3_level','r4_level','r5_level']].sum(axis=1)

In [117]:
train3['dire_level'] = train3[['d1_level','d2_level','d3_level','d4_level','d5_level']].sum(axis=1)

In [118]:
train3['radiant_lh'] = train3[['r1_lh','r2_lh','r3_lh','r4_lh','r5_lh']].sum(axis=1)

In [119]:
train3['dire_lh'] = train3[['d1_lh','d2_lh','d3_lh','d4_lh','d5_lh']].sum(axis=1)

In [120]:
train3['radiant_items'] = train3[['r1_items','r2_items','r3_items','r4_items','r5_items']].sum(axis=1)

In [121]:
train3['dire_items'] = train3[['d1_items','d2_items','d3_items','d4_items','d5_items']].sum(axis=1)

In [122]:
train3['radiant_agi_gain'] = train3[['r1_agi_gain','r2_agi_gain','r3_agi_gain','r4_agi_gain','r5_agi_gain']].sum(axis=1)

In [123]:
train3['dire_agi_gain'] = train3[['d1_agi_gain','d2_agi_gain','d3_agi_gain','d4_agi_gain','d5_agi_gain']].sum(axis=1)


In [124]:
train3['radiant_attack_range'] = train3[['r1_attack_range','r2_attack_range','r3_attack_range','r4_attack_range','r5_attack_range']].sum(axis=1)

In [125]:
train3['dire_attack_range'] = train3[['d1_attack_range','d2_attack_range','d3_attack_range','d4_attack_range','d5_attack_range']].sum(axis=1)

In [126]:
train3['radiant_attack_rate'] = train3[['r1_attack_rate','r2_attack_rate','r3_attack_rate','r4_attack_rate','r5_attack_rate']].sum(axis=1)

In [127]:
train3['dire_attack_rate'] = train3[['d1_attack_rate','d2_attack_rate','d3_attack_rate','d4_attack_rate','d5_attack_rate']].sum(axis=1)

In [128]:
train3['radiant_base_agi'] = train3[['r1_base_agi','r2_base_agi','r3_base_agi','r4_base_agi','r5_base_agi']].sum(axis=1)

In [129]:
train3['dire_base_agi'] = train3[['d1_base_agi','d2_base_agi','d3_base_agi','d4_base_agi','d5_base_agi']].sum(axis=1)

In [130]:
train3['radiant_base_armor'] = train3[['r1_base_armor','r2_base_armor','r3_base_armor','r4_base_armor','r5_base_armor']].sum(axis=1)

In [131]:
train3['dire_base_armor'] = train3[['d1_base_armor','d2_base_armor','d3_base_armor','d4_base_armor','d5_base_armor']].sum(axis=1)

In [132]:
train3['radiant_base_attack_max'] = train3[['r1_base_attack_max','r2_base_attack_max','r3_base_attack_max','r4_base_attack_max','r5_base_attack_max']].sum(axis=1)

In [133]:
train3['dire_base_attack_max'] = train3[['d1_base_attack_max','d2_base_attack_max','d3_base_attack_max','d4_base_attack_max','d5_base_attack_max']].sum(axis=1)

In [134]:
train3['radiant_base_attack_min'] = train3[['r1_base_attack_min','r2_base_attack_min','r3_base_attack_min','r4_base_attack_min','r5_base_attack_min']].sum(axis=1)



In [135]:
train3['dire_base_attack_min'] = train3[['d1_base_attack_min','d2_base_attack_min','d3_base_attack_min','d4_base_attack_min','d5_base_attack_min']].sum(axis=1)


In [136]:
train3['radiant_base_int'] = train3[['r1_base_int','r2_base_int','r3_base_int','r4_base_int','r5_base_int']].sum(axis=1)


In [137]:
train3['dire_base_int'] = train3[['d1_base_int','d2_base_int','d3_base_int','d4_base_int','d5_base_int']].sum(axis=1)


In [138]:
train3['radiant_base_str'] = train3[['r1_base_str','r2_base_str','r3_base_str','r4_base_str','r5_base_str']].sum(axis=1)


In [139]:
train3['dire_base_str'] = train3[['d1_base_str','d2_base_str','d3_base_str','d4_base_str','d5_base_str']].sum(axis=1)


In [140]:
train3['radiant_int_gain'] = train3[['r1_int_gain','r2_int_gain','r3_int_gain','r4_int_gain','r5_int_gain']].sum(axis=1)


In [141]:
train3['dire_int_gain'] = train3[['d1_int_gain','d2_int_gain','d3_int_gain','d4_int_gain','d5_int_gain']].sum(axis=1)


In [142]:
train3['radiant_legs'] = train3[['r1_legs','r2_legs','r3_legs','r4_legs','r5_legs']].sum(axis=1)


In [143]:
train3['dire_legs'] = train3[['d1_legs','d2_legs','d3_legs','d4_legs','d5_legs']].sum(axis=1)


In [144]:
train3['radiant_move_speed'] = train3[['r1_move_speed','r2_move_speed','r3_move_speed','r4_move_speed','r5_move_speed']].sum(axis=1)


In [145]:
train3['dire_move_speed'] = train3[['d1_move_speed','d2_move_speed','d3_move_speed','d4_move_speed','d5_move_speed']].sum(axis=1)


In [146]:
train3['radiant_primary_attr'] = train3[['r1_primary_attr','r2_primary_attr','r3_primary_attr','r4_primary_attr','r5_primary_attr']].sum(axis=1)


In [147]:
train3['dire_primary_attr'] = train3[['d1_primary_attr','d2_primary_attr','d3_primary_attr','d4_primary_attr','d5_primary_attr']].sum(axis=1)


In [148]:
train3['radiant_projectile_speed'] = train3[['r1_projectile_speed','r2_projectile_speed','r3_projectile_speed','r4_projectile_speed','r5_projectile_speed']].sum(axis=1)


In [149]:
train3['dire_projectile_speed'] = train3[['d1_projectile_speed','d2_projectile_speed','d3_projectile_speed','d4_projectile_speed','d5_projectile_speed']].sum(axis=1)


In [150]:
train3['radiant_str_gain'] = train3[['r1_str_gain','r2_str_gain','r3_str_gain','r4_str_gain','r5_str_gain']].sum(axis=1)


In [151]:
train3['dire_str_gain'] = train3[['d1_str_gain','d2_str_gain','d3_str_gain','d4_str_gain','d5_str_gain']].sum(axis=1)


In [152]:
train3['radiant_turn_rate'] = train3[['r1_turn_rate','r2_turn_rate','r3_turn_rate','r4_turn_rate','r5_turn_rate']].sum(axis=1)


In [153]:
train3['dire_turn_rate'] = train3[['d1_turn_rate','d2_turn_rate','d3_turn_rate','d4_turn_rate','d5_turn_rate']].sum(axis=1)


In [154]:
for cols in train3.columns:
    print("'"+cols+"',")

'Unnamed: 0',
'r1_hero',
'r1_level',
'r1_xp',
'r1_gold',
'r1_lh',
'r1_kills',
'r1_deaths',
'r1_items',
'r2_hero',
'r2_level',
'r2_xp',
'r2_gold',
'r2_lh',
'r2_kills',
'r2_deaths',
'r2_items',
'r3_hero',
'r3_level',
'r3_xp',
'r3_gold',
'r3_lh',
'r3_kills',
'r3_deaths',
'r3_items',
'r4_hero',
'r4_level',
'r4_xp',
'r4_gold',
'r4_lh',
'r4_kills',
'r4_deaths',
'r4_items',
'r5_hero',
'r5_level',
'r5_xp',
'r5_gold',
'r5_lh',
'r5_kills',
'r5_deaths',
'r5_items',
'd1_hero',
'd1_level',
'd1_xp',
'd1_gold',
'd1_lh',
'd1_kills',
'd1_deaths',
'd1_items',
'd2_hero',
'd2_level',
'd2_xp',
'd2_gold',
'd2_lh',
'd2_kills',
'd2_deaths',
'd2_items',
'd3_hero',
'd3_level',
'd3_xp',
'd3_gold',
'd3_lh',
'd3_kills',
'd3_deaths',
'd3_items',
'd4_hero',
'd4_level',
'd4_xp',
'd4_gold',
'd4_lh',
'd4_kills',
'd4_deaths',
'd4_items',
'd5_hero',
'd5_level',
'd5_xp',
'd5_gold',
'd5_lh',
'd5_kills',
'd5_deaths',
'd5_items',
'first_blood_time',
'first_blood_team',
'first_blood_player1',
'first_blood_player2',
'radiant_b

In [155]:
teamTrain = train3[['first_blood_time',
'first_blood_team',
'first_blood_player1',
'first_blood_player2',
'radiant_bottle_time',
'radiant_courier_time',
'radiant_flying_courier_time',
'radiant_tpscroll_count',
'radiant_boots_count',
'radiant_ward_observer_count',
'radiant_ward_sentry_count',
'radiant_first_ward_time',
'radiant_total_gold',
'radiant_total_xp',
'radiant_total_kills',
'radiant_total_deaths',
'dire_bottle_time',
'dire_courier_time',
'dire_flying_courier_time',
'dire_tpscroll_count',
'dire_boots_count',
'dire_ward_observer_count',
'dire_ward_sentry_count',
'dire_first_ward_time',
'dire_total_gold',
'dire_total_xp',
'dire_total_kills',
'dire_total_deaths','radiant_level',
'dire_level',
'radiant_lh',
'dire_lh',
'radiant_items',
'dire_items',
'radiant_agi_gain',
'dire_agi_gain',
'radiant_attack_range',
'dire_attack_range',
'radiant_attack_rate',
'dire_attack_rate',
'radiant_base_agi',
'dire_base_agi',
'radiant_base_armor',
'dire_base_armor',
'radiant_base_attack_max',
'dire_base_attack_max',
'radiant_base_attack_min',
'dire_base_attack_min',
'radiant_base_int',
'dire_base_int',
'radiant_base_str',
'dire_base_str',
'radiant_int_gain',
'dire_int_gain',
'radiant_legs',
'dire_legs',
'radiant_move_speed',
'dire_move_speed',
'radiant_projectile_speed',
'dire_projectile_speed',
'radiant_str_gain',
'dire_str_gain',
'radiant_turn_rate',
'dire_turn_rate',
'radiant_win']]

In [156]:
teamTrain.head()

,first_blood_time,first_blood_team,first_blood_player1,first_blood_player2,radiant_bottle_time,radiant_courier_time,radiant_flying_courier_time,radiant_tpscroll_count,radiant_boots_count,radiant_ward_observer_count,...,dire_legs,radiant_move_speed,dire_move_speed,radiant_projectile_speed,dire_projectile_speed,radiant_str_gain,dire_str_gain,radiant_turn_rate,dire_turn_rate,radiant_win
0,NaN,NaN,NaN,NaN,165.0,-55.0,NaN,6,4,1,...,8,1485,1440,4500,5400,14.1,12.0,3.0,2.7,1
1,166.0,0.0,1.0,6.0,NaN,-54.0,205.0,0,3,2,...,10,1480,1480,5750,7050,14.3,12.8,2.8,2.9,0
2,159.0,0.0,2.0,7.0,119.0,-71.0,NaN,0,4,2,...,12,1505,1485,5200,3400,12.9,17.5,2.8,3.0,1
3,184.0,1.0,8.0,4.0,160.0,-55.0,216.0,0,2,2,...,4,1470,1480,4475,3600,11.4,12.8,3.4,2.6,0
4,132.0,1.0,9.0,4.0,167.0,NaN,NaN,3,3,0,...,10,1475,1505,4950,6600,13.9,13.9,3.0,2.9,0


In [157]:
teamTrain['first_blood_time'].fillna((teamTrain['first_blood_time'].mean()), inplace=True)
teamTrain['radiant_bottle_time'].fillna((teamTrain['radiant_bottle_time'].mean()), inplace=True)
teamTrain['radiant_courier_time'].fillna((teamTrain['radiant_courier_time'].mean()), inplace=True)
teamTrain['radiant_flying_courier_time'].fillna((teamTrain['radiant_flying_courier_time'].mean()), inplace=True)
teamTrain['radiant_first_ward_time'].fillna((teamTrain['radiant_first_ward_time'].mean()), inplace=True)
teamTrain['dire_bottle_time'].fillna((teamTrain['dire_bottle_time'].mean()), inplace=True)
teamTrain['dire_courier_time'].fillna((teamTrain['dire_courier_time'].mean()), inplace=True)
teamTrain['dire_flying_courier_time'].fillna((teamTrain['dire_flying_courier_time'].mean()), inplace=True)
teamTrain['dire_first_ward_time'].fillna((teamTrain['dire_first_ward_time'].mean()), inplace=True)

//anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [158]:
teamTrain['first_blood_team'].fillna(999, inplace=True)
teamTrain['first_blood_player1'].fillna(999, inplace=True)
teamTrain['first_blood_player2'].fillna(999, inplace=True)

In [159]:
X=teamTrain[['first_blood_time',
'first_blood_team',
'first_blood_player1',
'first_blood_player2',
'radiant_bottle_time',
'radiant_courier_time',
'radiant_flying_courier_time',
'radiant_tpscroll_count',
'radiant_boots_count',
'radiant_ward_observer_count',
'radiant_ward_sentry_count',
'radiant_first_ward_time',
'radiant_total_gold',
'radiant_total_xp',
'radiant_total_kills',
'radiant_total_deaths',
'dire_bottle_time',
'dire_courier_time',
'dire_flying_courier_time',
'dire_tpscroll_count',
'dire_boots_count',
'dire_ward_observer_count',
'dire_ward_sentry_count',
'dire_first_ward_time',
'dire_total_gold',
'dire_total_xp',
'dire_total_kills',
'dire_total_deaths','radiant_level',
'dire_level',
'radiant_lh',
'dire_lh',
'radiant_items',
'dire_items',
'radiant_agi_gain',
'dire_agi_gain',
'radiant_attack_range',
'dire_attack_range',
'radiant_attack_rate',
'dire_attack_rate',
'radiant_base_agi',
'dire_base_agi',
'radiant_base_armor',
'dire_base_armor',
'radiant_base_attack_max',
'dire_base_attack_max',
'radiant_base_attack_min',
'dire_base_attack_min',
'radiant_base_int',
'dire_base_int',
'radiant_base_str',
'dire_base_str',
'radiant_int_gain',
'dire_int_gain',
'radiant_legs',
'dire_legs',
'radiant_move_speed',
'dire_move_speed',
'radiant_projectile_speed',
'dire_projectile_speed',
'radiant_str_gain',
'dire_str_gain',
'radiant_turn_rate',
'dire_turn_rate']]

y=teamTrain['radiant_win']

In [160]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)

In [161]:
clf=RandomForestClassifier(n_estimators=100)

In [162]:
clf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [163]:
y_pred=clf.predict(X_test)

In [164]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.6154894306547838


In [165]:
feature_imp = pd.Series(clf.feature_importances_,index=['first_blood_time',
'first_blood_team',
'first_blood_player1',
'first_blood_player2',
'radiant_bottle_time',
'radiant_courier_time',
'radiant_flying_courier_time',
'radiant_tpscroll_count',
'radiant_boots_count',
'radiant_ward_observer_count',
'radiant_ward_sentry_count',
'radiant_first_ward_time',
'radiant_total_gold',
'radiant_total_xp',
'radiant_total_kills',
'radiant_total_deaths',
'dire_bottle_time',
'dire_courier_time',
'dire_flying_courier_time',
'dire_tpscroll_count',
'dire_boots_count',
'dire_ward_observer_count',
'dire_ward_sentry_count',
'dire_first_ward_time',
'dire_total_gold',
'dire_total_xp',
'dire_total_kills',
'dire_total_deaths','radiant_level',
'dire_level',
'radiant_lh',
'dire_lh',
'radiant_items',
'dire_items',
'radiant_agi_gain',
'dire_agi_gain',
'radiant_attack_range',
'dire_attack_range',
'radiant_attack_rate',
'dire_attack_rate',
'radiant_base_agi',
'dire_base_agi',
'radiant_base_armor',
'dire_base_armor',
'radiant_base_attack_max',
'dire_base_attack_max',
'radiant_base_attack_min',
'dire_base_attack_min',
'radiant_base_int',
'dire_base_int',
'radiant_base_str',
'dire_base_str',
'radiant_int_gain',
'dire_int_gain',
'radiant_legs',
'dire_legs',
'radiant_move_speed',
'dire_move_speed',
'radiant_projectile_speed',
'dire_projectile_speed',
'radiant_str_gain',
'dire_str_gain',
'radiant_turn_rate',
'dire_turn_rate']).sort_values(ascending=False)
feature_imp

radiant_total_gold             0.038725
dire_total_gold                0.038376
radiant_total_xp               0.031147
dire_total_xp                  0.029533
dire_lh                        0.021409
radiant_lh                     0.020688
first_blood_time               0.019796
dire_attack_range              0.019590
dire_first_ward_time           0.019438
radiant_first_ward_time        0.019418
radiant_attack_range           0.019329
dire_int_gain                  0.019196
radiant_bottle_time            0.019119
radiant_int_gain               0.019107
dire_bottle_time               0.019066
dire_base_int                  0.019031
radiant_agi_gain               0.019014
dire_projectile_speed          0.018970
dire_agi_gain                  0.018863
radiant_courier_time           0.018863
radiant_projectile_speed       0.018827
radiant_base_int               0.018822
dire_courier_time              0.018609
dire_str_gain                  0.018471
radiant_base_attack_max        0.018372


In [166]:
for i in feature_imp:
    print(i)

0.038725457947432956
0.03837649101723787
0.03114748151872875
0.029533353641779895
0.021409171360435995
0.020688475461268806
0.01979613111666903
0.01959015724244391
0.019437645727205223
0.019418337060395755
0.01932926186136489
0.019195746119161982
0.01911918453075763
0.019107062764836515
0.01906598121755195
0.019031293339601206
0.01901386064549244
0.018969578995617457
0.01886319269388863
0.018862961429076588
0.018827131829301343
0.01882156119145395
0.01860934378323437
0.01847059855147153
0.018372351588996676
0.018263542153041875
0.018254347133343573
0.018023270139855184
0.018021685754933823
0.017865136640727034
0.01782996040216701
0.017711729467091805
0.017567526053728525
0.01628951077734125
0.01606406230065278
0.016041441471228237
0.01601530378101029
0.014292684083076615
0.014204439190159974
0.01415837268766459
0.01402042241766619
0.013231403472168566
0.013183256087417727
0.011498825209504989
0.01117482701673716
0.01091433522751152
0.010783351630493798
0.010394567991656571
0.0101413725

identify features that combine to 80% feature importance

In [167]:
X=teamTrain[['first_blood_time',
'radiant_bottle_time',
'radiant_courier_time',
'radiant_flying_courier_time',
'radiant_first_ward_time',
'radiant_total_gold',
'radiant_total_xp',
'dire_bottle_time',
'dire_courier_time',
'dire_flying_courier_time',
'dire_first_ward_time',
'dire_total_gold',
'dire_total_xp',
'radiant_lh',
'dire_lh',
'radiant_items',
'dire_items',
'radiant_agi_gain',
'dire_agi_gain',
'radiant_attack_range',
'dire_attack_range',
'radiant_base_agi',
'dire_base_agi',
'radiant_base_armor',
'dire_base_armor',
'radiant_base_attack_max',
'dire_base_attack_max',
'radiant_base_attack_min',
'dire_base_attack_min',
'radiant_base_int',
'dire_base_int',
'radiant_base_str',
'dire_base_str',
'radiant_int_gain',
'dire_int_gain',
'radiant_move_speed',
'dire_move_speed',
'radiant_projectile_speed',
'dire_projectile_speed',
'radiant_str_gain',
'dire_str_gain']]

y=teamTrain['radiant_win']

In [168]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)

In [169]:
clf=RandomForestClassifier(n_estimators=100)

In [170]:
clf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [171]:
y_pred=clf.predict(X_test)

In [172]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.6160786624438388


In [173]:
feature_imp = pd.Series(clf.feature_importances_,index=['first_blood_time',
'radiant_bottle_time',
'radiant_courier_time',
'radiant_flying_courier_time',
'radiant_first_ward_time',
'radiant_total_gold',
'radiant_total_xp',
'dire_bottle_time',
'dire_courier_time',
'dire_flying_courier_time',
'dire_first_ward_time',
'dire_total_gold',
'dire_total_xp',
'radiant_lh',
'dire_lh',
'radiant_items',
'dire_items',
'radiant_agi_gain',
'dire_agi_gain',
'radiant_attack_range',
'dire_attack_range',
'radiant_base_agi',
'dire_base_agi',
'radiant_base_armor',
'dire_base_armor',
'radiant_base_attack_max',
'dire_base_attack_max',
'radiant_base_attack_min',
'dire_base_attack_min',
'radiant_base_int',
'dire_base_int',
'radiant_base_str',
'dire_base_str',
'radiant_int_gain',
'dire_int_gain',
'radiant_move_speed',
'dire_move_speed',
'radiant_projectile_speed',
'dire_projectile_speed',
'radiant_str_gain',
'dire_str_gain']).sort_values(ascending=False)
feature_imp

dire_total_gold                0.046870
radiant_total_gold             0.044612
radiant_total_xp               0.039549
dire_total_xp                  0.039182
radiant_lh                     0.025327
first_blood_time               0.025204
dire_lh                        0.024949
dire_first_ward_time           0.024628
radiant_first_ward_time        0.024258
dire_attack_range              0.024153
dire_bottle_time               0.023880
radiant_attack_range           0.023845
radiant_int_gain               0.023663
radiant_bottle_time            0.023633
radiant_projectile_speed       0.023591
dire_int_gain                  0.023560
radiant_courier_time           0.023511
radiant_agi_gain               0.023342
dire_agi_gain                  0.023300
radiant_base_int               0.023273
dire_projectile_speed          0.023239
dire_courier_time              0.023037
dire_str_gain                  0.023012
dire_base_int                  0.023005
radiant_str_gain               0.022900


In [174]:
train5 = train3[['r1_hero',
'r2_hero',
'r3_hero',                 
'r4_hero',
'r5_hero',
'd1_hero',
'd2_hero',
'd3_hero',                 
'd4_hero',
'd5_hero',
'first_blood_time',
'first_blood_team',
'first_blood_player1',
'first_blood_player2',
'radiant_bottle_time',
'radiant_courier_time',
'radiant_flying_courier_time',
'radiant_tpscroll_count',
'radiant_boots_count',
'radiant_ward_observer_count',
'radiant_ward_sentry_count',
'radiant_first_ward_time',
'radiant_total_gold',
'radiant_total_xp',
'radiant_total_kills',
'radiant_total_deaths',
'dire_bottle_time',
'dire_courier_time',
'dire_flying_courier_time',
'dire_tpscroll_count',
'dire_boots_count',
'dire_ward_observer_count',
'dire_ward_sentry_count',
'dire_first_ward_time',
'dire_total_gold',
'dire_total_xp',
'dire_total_kills',
'dire_total_deaths','radiant_level',
'dire_level',
'radiant_lh',
'dire_lh',
'radiant_items',
'dire_items',
'radiant_agi_gain',
'dire_agi_gain',
'radiant_attack_range',
'dire_attack_range',
'radiant_attack_rate',
'dire_attack_rate',
'radiant_base_agi',
'dire_base_agi',
'radiant_base_armor',
'dire_base_armor',
'radiant_base_attack_max',
'dire_base_attack_max',
'radiant_base_attack_min',
'dire_base_attack_min',
'radiant_base_int',
'dire_base_int',
'radiant_base_str',
'dire_base_str',
'radiant_int_gain',
'dire_int_gain',
'radiant_legs',
'dire_legs',
'radiant_move_speed',
'dire_move_speed',
'radiant_projectile_speed',
'dire_projectile_speed',
'radiant_str_gain',
'dire_str_gain',
'radiant_turn_rate',
'dire_turn_rate',
'radiant_win']]

In [175]:
train5['first_blood_time'].fillna((train5['first_blood_time'].mean()), inplace=True)
train5['radiant_bottle_time'].fillna((train5['radiant_bottle_time'].mean()), inplace=True)
train5['radiant_courier_time'].fillna((train5['radiant_courier_time'].mean()), inplace=True)
train5['radiant_flying_courier_time'].fillna((train5['radiant_flying_courier_time'].mean()), inplace=True)
train5['radiant_first_ward_time'].fillna((train5['radiant_first_ward_time'].mean()), inplace=True)
train5['dire_bottle_time'].fillna((train5['dire_bottle_time'].mean()), inplace=True)
train5['dire_courier_time'].fillna((train5['dire_courier_time'].mean()), inplace=True)
train5['dire_flying_courier_time'].fillna((train5['dire_flying_courier_time'].mean()), inplace=True)
train5['dire_first_ward_time'].fillna((train5['dire_first_ward_time'].mean()), inplace=True)

//anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [176]:
train5['first_blood_team'].fillna(999, inplace=True)
train5['first_blood_player1'].fillna(999, inplace=True)
train5['first_blood_player2'].fillna(999, inplace=True)

In [177]:
X=train5[['r1_hero',
'r2_hero',
'r3_hero',                 
'r4_hero',
'r5_hero',
'd1_hero',
'd2_hero',
'd3_hero',                 
'd4_hero',
'd5_hero',
'first_blood_time',
'first_blood_team',
'first_blood_player1',
'first_blood_player2',
'radiant_bottle_time',
'radiant_courier_time',
'radiant_flying_courier_time',
'radiant_tpscroll_count',
'radiant_boots_count',
'radiant_ward_observer_count',
'radiant_ward_sentry_count',
'radiant_first_ward_time',
'radiant_total_gold',
'radiant_total_xp',
'radiant_total_kills',
'radiant_total_deaths',
'dire_bottle_time',
'dire_courier_time',
'dire_flying_courier_time',
'dire_tpscroll_count',
'dire_boots_count',
'dire_ward_observer_count',
'dire_ward_sentry_count',
'dire_first_ward_time',
'dire_total_gold',
'dire_total_xp',
'dire_total_kills',
'dire_total_deaths','radiant_level',
'dire_level',
'radiant_lh',
'dire_lh',
'radiant_items',
'dire_items',
'radiant_agi_gain',
'dire_agi_gain',
'radiant_attack_range',
'dire_attack_range',
'radiant_attack_rate',
'dire_attack_rate',
'radiant_base_agi',
'dire_base_agi',
'radiant_base_armor',
'dire_base_armor',
'radiant_base_attack_max',
'dire_base_attack_max',
'radiant_base_attack_min',
'dire_base_attack_min',
'radiant_base_int',
'dire_base_int',
'radiant_base_str',
'dire_base_str',
'radiant_int_gain',
'dire_int_gain',
'radiant_legs',
'dire_legs',
'radiant_move_speed',
'dire_move_speed',
'radiant_projectile_speed',
'dire_projectile_speed',
'radiant_str_gain',
'dire_str_gain',
'radiant_turn_rate',
'dire_turn_rate']]

In [178]:
y=train5['radiant_win']

In [179]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)

In [180]:
clf=RandomForestClassifier(n_estimators=100)

In [181]:
clf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [182]:
y_pred=clf.predict(X_test)

In [183]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.6116962510127422


In [184]:
feature_imp = pd.Series(clf.feature_importances_,index=['r1_hero',
'r2_hero',
'r3_hero',                 
'r4_hero',
'r5_hero',
'd1_hero',
'd2_hero',
'd3_hero',                 
'd4_hero',
'd5_hero',
'first_blood_time',
'first_blood_team',
'first_blood_player1',
'first_blood_player2',
'radiant_bottle_time',
'radiant_courier_time',
'radiant_flying_courier_time',
'radiant_tpscroll_count',
'radiant_boots_count',
'radiant_ward_observer_count',
'radiant_ward_sentry_count',
'radiant_first_ward_time',
'radiant_total_gold',
'radiant_total_xp',
'radiant_total_kills',
'radiant_total_deaths',
'dire_bottle_time',
'dire_courier_time',
'dire_flying_courier_time',
'dire_tpscroll_count',
'dire_boots_count',
'dire_ward_observer_count',
'dire_ward_sentry_count',
'dire_first_ward_time',
'dire_total_gold',
'dire_total_xp',
'dire_total_kills',
'dire_total_deaths','radiant_level',
'dire_level',
'radiant_lh',
'dire_lh',
'radiant_items',
'dire_items',
'radiant_agi_gain',
'dire_agi_gain',
'radiant_attack_range',
'dire_attack_range',
'radiant_attack_rate',
'dire_attack_rate',
'radiant_base_agi',
'dire_base_agi',
'radiant_base_armor',
'dire_base_armor',
'radiant_base_attack_max',
'dire_base_attack_max',
'radiant_base_attack_min',
'dire_base_attack_min',
'radiant_base_int',
'dire_base_int',
'radiant_base_str',
'dire_base_str',
'radiant_int_gain',
'dire_int_gain',
'radiant_legs',
'dire_legs',
'radiant_move_speed',
'dire_move_speed',
'radiant_projectile_speed',
'dire_projectile_speed',
'radiant_str_gain',
'dire_str_gain',
'radiant_turn_rate',
'dire_turn_rate']).sort_values(ascending=False)
feature_imp

dire_total_gold                0.033722
radiant_total_gold             0.033193
radiant_total_xp               0.029081
dire_total_xp                  0.027262
radiant_lh                     0.018585
dire_lh                        0.018033
first_blood_time               0.016455
radiant_attack_range           0.016230
dire_first_ward_time           0.016224
dire_attack_range              0.016183
radiant_first_ward_time        0.016177
radiant_bottle_time            0.016122
dire_int_gain                  0.016006
d4_hero                        0.015956
radiant_int_gain               0.015946
r3_hero                        0.015906
r4_hero                        0.015886
d2_hero                        0.015835
d3_hero                        0.015827
radiant_base_int               0.015807
r1_hero                        0.015805
dire_base_int                  0.015784
r2_hero                        0.015775
radiant_agi_gain               0.015757
r5_hero                        0.015722


In [185]:
for i in feature_imp:
    print(i)

0.03372226545642944
0.03319341776953195
0.029080587224017997
0.027262261555445633
0.01858519179212845
0.018033128172920864
0.01645522680073616
0.0162302229181848
0.016223737683642596
0.016183484696555755
0.016177149617153764
0.016121539419178224
0.016005592398419487
0.015955778676354625
0.015945705914011038
0.015905748348156445
0.015886122065148205
0.01583483165440995
0.01582698434496776
0.015806577519568276
0.015804831073347136
0.01578411871550449
0.01577467860718552
0.015756679193218267
0.015721507536203275
0.01569968590390903
0.015696870182982596
0.01569105529452174
0.015613847758010153
0.015600468717377958
0.01557098608336267
0.015556257058086187
0.015467450194682568
0.015453977019869664
0.015316416616344728
0.01531062966739941
0.015130990636911312
0.01503493805067921
0.014859262390434738
0.01484926732625898
0.014832090405606138
0.01469032391808103
0.014680651487239505
0.013322282527691296
0.013291201182161691
0.013249858575281872
0.013192989580362326
0.011814275874639554
0.0116807

feature reduction with train 5

In [186]:
X=train5[['r1_hero',
'r2_hero',
'r3_hero',                 
'r4_hero',
'r5_hero',
'd1_hero',
'd2_hero',
'd3_hero',                 
'd4_hero',
'd5_hero',
'first_blood_time',
'radiant_bottle_time',
'radiant_courier_time',
'radiant_flying_courier_time',
'radiant_first_ward_time',
'radiant_total_gold',
'radiant_total_xp',
'dire_bottle_time',
'dire_courier_time',
'dire_flying_courier_time',
'dire_first_ward_time',
'dire_total_gold',
'dire_total_xp',
'radiant_lh',
'dire_lh',
'radiant_items',
'dire_items',
'radiant_agi_gain',
'dire_agi_gain',
'radiant_attack_range',
'dire_attack_range',
'radiant_base_agi',
'dire_base_agi',
'radiant_base_attack_max',
'dire_base_attack_max',
'radiant_base_attack_min',
'dire_base_attack_min',
'radiant_base_int',
'dire_base_int',
'radiant_base_str',
'dire_base_str',
'radiant_int_gain',
'dire_int_gain',
'radiant_move_speed',
'dire_move_speed',
'radiant_projectile_speed',
'dire_projectile_speed',
'radiant_str_gain',
'dire_str_gain']]

In [187]:
y=train5['radiant_win']

In [188]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)

In [189]:
clf=RandomForestClassifier(n_estimators=100)

In [190]:
clf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [191]:
y_pred=clf.predict(X_test)

In [192]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.6143477940634897


In [193]:
feature_imp = pd.Series(clf.feature_importances_,index=['r1_hero',
'r2_hero',
'r3_hero',                 
'r4_hero',
'r5_hero',
'd1_hero',
'd2_hero',
'd3_hero',                 
'd4_hero',
'd5_hero',
'first_blood_time',
'radiant_bottle_time',
'radiant_courier_time',
'radiant_flying_courier_time',
'radiant_first_ward_time',
'radiant_total_gold',
'radiant_total_xp',
'dire_bottle_time',
'dire_courier_time',
'dire_flying_courier_time',
'dire_first_ward_time',
'dire_total_gold',
'dire_total_xp',
'radiant_lh',
'dire_lh',
'radiant_items',
'dire_items',
'radiant_agi_gain',
'dire_agi_gain',
'radiant_attack_range',
'dire_attack_range',
'radiant_base_agi',
'dire_base_agi',
'radiant_base_attack_max',
'dire_base_attack_max',
'radiant_base_attack_min',
'dire_base_attack_min',
'radiant_base_int',
'dire_base_int',
'radiant_base_str',
'dire_base_str',
'radiant_int_gain',
'dire_int_gain',
'radiant_move_speed',
'dire_move_speed',
'radiant_projectile_speed',
'dire_projectile_speed',
'radiant_str_gain',
'dire_str_gain']).sort_values(ascending=False)
feature_imp

dire_total_gold                0.043508
radiant_total_gold             0.040685
radiant_total_xp               0.033563
dire_total_xp                  0.032476
radiant_lh                     0.021642
dire_lh                        0.021338
first_blood_time               0.020631
dire_first_ward_time           0.020076
radiant_attack_range           0.020010
radiant_first_ward_time        0.019855
r3_hero                        0.019818
d4_hero                        0.019778
radiant_bottle_time            0.019771
dire_attack_range              0.019700
d5_hero                        0.019692
r4_hero                        0.019684
d3_hero                        0.019682
r5_hero                        0.019571
d1_hero                        0.019548
dire_bottle_time               0.019498
r1_hero                        0.019446
radiant_agi_gain               0.019444
r2_hero                        0.019437
dire_int_gain                  0.019391
dire_courier_time              0.019333
